In [ ]:
!git clone https://github.com/mnaufaldillah/Dynamic-Animated-Thumbnail.git

Cloning into 'Dynamic-Animated-Thumbnail'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (21/21), done.
Receiving objects: 100% (42/42), 7.29 KiB | 7.29 MiB/s, done.
Resolving deltas: 100% (17/17), done.
remote: Total 42 (delta 17), reused 33 (delta 11), pack-reused 0 (from 0)


In [ ]:
# Installing fastapi
!pip install fastapi nest-asyncio pyngrok uvicorn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 3.7 MB/s eta 0:00:00


In [ ]:
# Installing pytubefix
!pip install pytubefix

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 3.7 MB/s eta 0:00:00


In [ ]:
#Installing pytube
!pip install pytube

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.3 MB/s eta 0:00:00


In [ ]:
# Import Library

from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.applications import ResNet50
from fastapi.staticfiles import StaticFiles
from fastapi.responses import HTMLResponse, FileResponse
from fastapi import FastAPI, UploadFile, File, Request
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
from pyngrok import ngrok
import uvicorn
from moviepy.editor import *
from pytubefix.cli import on_progress
from pytubefix import YouTube
import pytube
import numpy as np
import cv2
import os

  if event.key is 'enter':



In [ ]:
# Load Thre Pre-Trained Model, this time with ResNet50 Architecture
model = ResNet50(weights='imagenet', include_top=False)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
# Function to Download Video from YouTube's URL
def download_video(video_url: str, output_path: str) -> str:
  try:
    #Create Youtube Object
    yt = YouTube(video_url, on_progress_callback=on_progress)
    print(yt.title)
    stream = yt.streams.filter(adaptive=True).filter(mime_type='video/webm').first()

    if stream is None:
      raise Exception("No valid stream found to download")

    os.makedirs(output_path, exist_ok=True) #Checking if the directory exists
    file_path = stream.download(output_path=output_path)
    return file_path

  except pytube.exceptions.RegexMatchError:
    raise Exception("Invalid YouTube URL")
  except pytube.exceptions.VideoUnavailable:
    raise Exception("Video is unavailable")
  except pytube.exceptions.AgeRestrictedError:
    raise Exception("Video is age-restricted")
  except Exception as e:
    raise Exception(f"Internal server error: {str(e)}")

In [ ]:
# Function to Extract Video's keyframes
def extract_keyframes(video_path: str, frame_rate: int = 1) -> list:
  # Extract frames from video at specific frame rate
  if not os.path.exists(video_path):
    print("Video file not found")

  cap = cv2.VideoCapture(video_path)
  frame_list = []
  fps = cap.get(cv2.CAP_PROP_FPS) # Video FPS
  success, frame = cap.read()
  count = 0
  while success:
    # Select frames based on frame rate
    if count % int(fps / frame_rate) == 0:
      frame_list.append(frame)
    success, frame = cap.read()
    count += 1

  cap.release()
  return frame_list

In [ ]:
# Preprocessing Frame Function
def preprocess_frame(frame):
  # Preprocess the frame for the CNN ResNet50 model.
  # Resizing the frame and convert it into a batch of size 1.
  frame_resized = cv2.resize(frame, (224, 224)) # Resizing to 224*224
  frame_array = np.expand_dims(frame, axis=0)
  return preprocess_input(frame_array)

In [ ]:
# Applyiing ResNet 50 Model to Extract Features and Identify best frames
# that represents the video
def extract_features(frames):
  # Using ResNet50 model to extract features
  features = []
  for frame in frames:
    frame_preprocessed = preprocess_frame(frame)
    feature = model.predict(frame_preprocessed)
    features.append(np.mean(feature)) # Meaon of Extracted Features (Simplified Selection Criterion)
  return features

In [ ]:
# Selecting the best keyframes
def select_keyframes(frames, features, num_keyframes=10):
  # Select the keyframes based on the highest-scoring features.
  # Select top N keyframes.
  keyframes_indices = np.argsort(features)[-num_keyframes:]
  keyframes = [frames[i] for i in keyframes_indices]
  return keyframes

In [ ]:
# Generate Animated GIF as Thumnail
def generate_animated_gif(frames: list, output_gif: str) -> None:
  # Generate Animated GIF from selected keyframes.
  clip = ImageSequenceClip([cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) for frame in frames], fps=6) # Adjust the FPS
  print(output_gif)
  clip.write_gif(output_gif, fps=6)

In [ ]:
!mkdir Dynamic-Animated-Thumbnail/thumbnails

In [ ]:
!mkdir Dynamic-Animated-Thumbnail/videos

In [ ]:
# Setting up FastAPI
app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

# Mount to Static Directory Folder to serve CSS file
app.mount("/static", StaticFiles(directory="Dynamic-Animated-Thumbnail/static"), name="static")
app.mount("/thumbnails", StaticFiles(directory="Dynamic-Animated-Thumbnail/thumbnails"), name="thumbnails")
app.mount("/videos", StaticFiles(directory="Dynamic-Animated-Thumbnail/videos"), name="videos")

In [ ]:
@app.get("/")
async def main_page():
  with open("./Dynamic-Animated-Thumbnail/templates/index.html", "r") as f:
    return HTMLResponse(content=f.read())

In [ ]:
@app.post("/generate-thumbnail/")
async def generate_thumbnail(request: Request):
  data = await request.json()
  video_url = data.get("video_url")

  if not video_url:
    return {"error": "Video URL is required"}

  # Download Video from Youtube URL
  video_path = download_video(video_url, "./Dynamic-Animated-Thumbnail/videos")
  video_filename = os.path.basename(video_path)

  # Extract Keyframes from Video
  frames = extract_keyframes(video_path, frame_rate=1)
  print(f"Number of frames extracted: {len(frames)}")

  features = extract_features(frames)
  print(f"Number of features extracted: {len(features)}")

  # Select Best Keyframes
  keyframes = select_keyframes(frames, features, num_keyframes=10)
  print(f"Number of keyframes selected: {len(keyframes)}")

  # Generate Animated GIF
  output_gif_path = f"./Dynamic-Animated-Thumbnail/thumbnails/{os.path.basename(video_path)}.gif"
  generate_animated_gif(keyframes, output_gif_path)

  print(FileResponse(output_gif_path, media_type="image/gif"))

  output_gif_url = f"thumbnails/{os.path.basename(video_path)}.gif"

  print(output_gif_url)

  # Return Animated GIF as Response
  return {"thumbnail_url": output_gif_url, "video_filename": video_filename}

In [ ]:
ngrok.set_auth_token("2l31H1PTpYemZO0JqBMN3aDBsfI_6dT29ZVvqB8XZMRdPZeVm")

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)

nest_asyncio.apply()

uvicorn.run(app, port=8000)

INFO:     Started server process [357]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://ec76-104-199-127-199.ngrok-free.app
INFO:     2001:448a:50e0:d0d4:65e7:43d:139d:34cd:0 - "GET / HTTP/1.1" 200 OK
INFO:     2001:448a:50e0:d0d4:65e7:43d:139d:34cd:0 - "GET /static/style.css HTTP/1.1" 200 OK
INFO:     2001:448a:50e0:d0d4:65e7:43d:139d:34cd:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
Cat fight extreme sound 😱
Number of frames extracted: 71
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━

thumbnails/Cat fight extreme sound 😱.mp4.gif
INFO:     2001:448a:50e0:d0d4:65e7:43d:139d:34cd:0 - "POST /generate-thumbnail/ HTTP/1.1" 200 OK
INFO:     2001:448a:50e0:d0d4:65e7:43d:139d:34cd:0 - "GET /thumbnails/Cat%20fight%20extreme%20sound%20%F0%9F%98%B1.mp4.gif HTTP/1.1" 200 OK
INFO:     2001:448a:50e0:d0d4:65e7:43d:139d:34cd:0 - "GET /videos/Cat%20fight%20extreme%20sound%20%F0%9F%98%B1.mp4 HTTP/1.1" 206 Partial Content
INFO:     2001:448a:50e0:d0d4:65e7:43d:139d:34cd:0 - "GET /videos/Cat%20fight%20extreme%20sound%20%F0%9F%98%B1.mp4 HTTP/1.1" 206 Partial Content
WOW!!! Yamaha Team Made Big Improvement at Barcelona Test 2025 MotoGP Season with Fabio Quartararo
Number of frames extracted: 241
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/

thumbnails/WOW!!! Yamaha Team Made Big Improvement at Barcelona Test 2025 MotoGP Season with Fabio Quartararo.mp4.gif
INFO:     2001:448a:50e0:d0d4:65e7:43d:139d:34cd:0 - "POST /generate-thumbnail/ HTTP/1.1" 200 OK
INFO:     2001:448a:50e0:d0d4:65e7:43d:139d:34cd:0 - "GET /thumbnails/WOW%21%21%21%20Yamaha%20Team%20Made%20Big%20Improvement%20at%20Barcelona%20Test%202025%20MotoGP%20Season%20with%20Fabio%20Quartararo.mp4.gif HTTP/1.1" 200 OK
INFO:     2001:448a:50e0:d0d4:65e7:43d:139d:34cd:0 - "GET /videos/WOW%21%21%21%20Yamaha%20Team%20Made%20Big%20Improvement%20at%20Barcelona%20Test%202025%20MotoGP%20Season%20with%20Fabio%20Quartararo.mp4 HTTP/1.1" 206 Partial Content
INFO:     2001:448a:50e0:d0d4:65e7:43d:139d:34cd:0 - "GET /videos/WOW%21%21%21%20Yamaha%20Team%20Made%20Big%20Improvement%20at%20Barcelona%20Test%202025%20MotoGP%20Season%20with%20Fabio%20Quartararo.mp4 HTTP/1.1" 206 Partial Content
INFO:     2001:448a:50e0:d0d4:65e7:43d:139d:34cd:0 - "GET /videos/WOW%21%21%21%20Yamaha%20T